# [Fe/H] Metallicity

In [9]:
from matplotlib import pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import pandas as pd
from scipy.stats import gaussian_kde

In [2]:
from auriga.snapshot import Snapshot
from auriga.images import figure_setup, set_axs_configuration
from auriga.settings import Settings

In [3]:
figure_setup()

In [4]:
def read_data(simulations: list) -> pd.DataFrame:
    """
    Return a Pandas data frame with abundance values for each galaxy
    and each component.

    Parameters
    ----------
    simulations : list
        A list with all the simulations.

    Returns
    -------
    pd.DataFrame
        A data frame with the results.
    """
    settings = Settings()
    df = pd.DataFrame(
        index=list(range(30 * 5)),
        columns=["Simulation",
                 "Fe/H_Median", "Fe/H_Perc16", "Fe/H_Perc84",
                 "O/H_Median", "O/H_Perc16", "O/H_Perc84",
                 "O/Fe_Median", "O/Fe_Perc16", "O/Fe_Perc84",
                 "Component"])

    k = 0
    for i, simulation in enumerate(simulations):
        s = Snapshot(simulation=simulation,
                     loadonlytype=[0, 1, 2, 3, 4, 5])
        s.add_circularity()
        s.add_reference_to_potential()
        s.add_normalized_potential()
        s.tag_particles_by_region(disc_std_circ=1.0,
                                  disc_min_circ=0.4,
                                  cold_disc_delta_circ=0.25,
                                  bulge_max_specific_energy=-0.75)
        s.add_metal_abundance(of="Fe", to="H")
        s.add_metal_abundance(of="O", to="H")
        s.add_metal_abundance(of="O", to="Fe")

        is_real_star = (s.type == 4) & (s.stellar_formation_time > 0)
        is_main_obj = (s.halo == s.halo_idx) & (s.subhalo == s.subhalo_idx)

        for j, component in enumerate(settings.components + ["All"]):
            if component == "All":
                is_region = np.ones(s.region_tag.shape, dtype=bool)
            else:
                is_region = s.region_tag == j
            df.iloc[k, 0] = simulation
            df.iloc[k, 1] = np.nanmedian(
                s.metal_abundance["Fe/H"][is_real_star \
                    & is_main_obj & is_region])
            df.iloc[k, 2] = np.nanpercentile(
                s.metal_abundance["Fe/H"][is_real_star \
                    & is_main_obj & is_region], 16)
            df.iloc[k, 3] = np.nanpercentile(
                s.metal_abundance["Fe/H"][is_real_star \
                    & is_main_obj & is_region], 84)
            df.iloc[k, 4] = np.nanmedian(
                s.metal_abundance["O/H"][is_real_star \
                    & is_main_obj & is_region])
            df.iloc[k, 5] = np.nanpercentile(
                s.metal_abundance["O/H"][is_real_star \
                    & is_main_obj & is_region], 16)
            df.iloc[k, 6] = np.nanpercentile(
                s.metal_abundance["O/H"][is_real_star \
                    & is_main_obj & is_region], 84)
            df.iloc[k, 7] = np.nanmedian(
                s.metal_abundance["O/Fe"][is_real_star \
                    & is_main_obj & is_region])
            df.iloc[k, 8] = np.nanpercentile(
                s.metal_abundance["O/Fe"][is_real_star \
                    & is_main_obj & is_region], 16)
            df.iloc[k, 9] = np.nanpercentile(
                s.metal_abundance["O/Fe"][is_real_star \
                    & is_main_obj & is_region], 84)
            df.iloc[k, 10] = -1 if component == "All" else j
            k += 1

    return df

In [5]:
settings = Settings()
originals = [f"au{i}_or_l4_s127" for i in settings.galaxies]
df = read_data(simulations=originals)

In [6]:
# Calculate the error bars
xerr = np.abs(df[["Fe/H_Perc16", "Fe/H_Perc84"]].to_numpy() \
    - np.column_stack((df["Fe/H_Median"].to_numpy(),
                    df["Fe/H_Median"].to_numpy())))
yerr = np.abs(df[["O/Fe_Perc16", "O/Fe_Perc84"]].to_numpy() \
    - np.column_stack((df["O/Fe_Median"].to_numpy(),
                    df["O/Fe_Median"].to_numpy())))

## median([Fe/H]) Abundance Histogram

In [189]:
fig = plt.figure(figsize=(2.0, 2.0))
gs = fig.add_gridspec(nrows=1, ncols=1, hspace=0.0, wspace=0.0)
ax = gs.subplots(sharex=True, sharey=True)

ax.grid(True, ls='-', lw=0.25, c='silver')
ax.tick_params(which='both', direction="in")
ax.set_xlim(-0.7, 0.7)
ax.set_ylim(0, 8)
ax.set_xticks([-0.5, 0, 0.5])
ax.set_yticks([0, 2, 4, 6, 8])
ax.set_xlabel('median([Fe/H])')
ax.set_ylabel('PDF')
ax.set_axisbelow(True)

ax.hist(df["Fe/H_Median"][df["Component"] == -1], bins=15, range=(-0.5, 0.5),
    histtype="stepfilled", color="k", alpha=0.25, lw=0, density=True,
    label="All")
ax.hist(df["Fe/H_Median"][df["Component"] == 0], bins=15, range=(-0.5, 0.5),
    histtype="step", color="tab:blue", alpha=1, density=True,
    label="Halo")
ax.hist(df["Fe/H_Median"][df["Component"] == 1], bins=15, range=(-0.5, 0.5),
    histtype="step", color="tab:green", alpha=1, density=True,
    label="Bulge")
ax.hist(df["Fe/H_Median"][df["Component"] == 2], bins=15, range=(-0.5, 0.5),
    histtype="step", color="tab:red", alpha=1, density=True,
    label="Cold Disc")
ax.hist(df["Fe/H_Median"][df["Component"] == 3], bins=15, range=(-0.5, 0.5),
    histtype="step", color="tab:orange", alpha=1, density=True,
    label="Warm Disc")

ax.legend(loc="upper left", framealpha=0, fontsize=5.0)

fig.savefig("../images/metallicity/FeH_histogram_originals_s127.pdf")
plt.close(fig)

## median([O/Fe]) vs median([Fe/H]) Scatter

In [37]:
fig, axs = plt.subplots(
    figsize=(7.4, 2.0), nrows=1, ncols=4, sharey=True, sharex=True,
    gridspec_kw={"hspace": 0.0, "wspace": 0.0})

for ax in axs.flat:
    ax.grid(True, ls='-', lw=0.25, c="gainsboro")
    ax.tick_params(which='both', direction="in")
    ax.set_xlim(-1.0, 0.8)
    ax.set_ylim(0.18, 0.32)
    ax.set_xticks([-0.8, -0.4, 0, 0.4])
    ax.set_yticks([0.20, 0.22, 0.24, 0.26, 0.28, 0.3])
    if ax == axs.flat[0]:
        ax.set_ylabel('median([O/Fe])')
    ax.set_xlabel('median([Fe/H])')
    ax.set_axisbelow(True)

markers = ["o", "v", "^", "d"]
colors = list(settings.component_colors.values())
labels = list(settings.component_labels.values())

for i, ax in enumerate(axs.flat):
    for j in range(4):
        label = labels[j] if j == i else None
        color = colors[j] if j == i else "silver"
        zorder = 10 if j == i else 5
        ax.scatter(
            df["Fe/H_Median"][df["Component"] == j],
            df["O/Fe_Median"][df["Component"] == j],
            c=color, label=label, zorder=zorder,
            s=20, linewidths=0.4, edgecolors="white", marker=markers[j])
        
        if j == i:
            fe_pdf = gaussian_kde(
                df["Fe/H_Median"][df["Component"] == j].to_numpy(
                    dtype=np.float64))
            pdf_x = np.linspace(-1.0, 0.6, 100)
            pdf_y = fe_pdf(pdf_x) / np.max(fe_pdf(pdf_x)) * 0.02 \
                + ax.get_ylim()[0]
            ax.fill_between(
                x=pdf_x, y1=-1.0, y2=pdf_y, edgecolor=color,
                facecolor=mcolors.TABLEAU_COLORS[color] + "30")
            
            o_pdf = gaussian_kde(
                df["O/Fe_Median"][df["Component"] == j].to_numpy(
                    dtype=np.float64))
            pdf_y = np.linspace(0.18, 0.3, 100)
            pdf_x = o_pdf(pdf_y) / np.max(o_pdf(pdf_y)) * 0.3
            ax.fill_betweenx(
                y=pdf_y, x1=-pdf_x + 0.8, x2=0.8, edgecolor=color,
                facecolor=mcolors.TABLEAU_COLORS[color] + "30")

    ax.legend(loc="upper left", framealpha=0, fontsize=7.5)

fig.savefig("../images/metallicity/OFe_vs_FeH_originals_s127.pdf")
plt.close(fig)

## median([O/Fe]) vs median([Fe/H]) Scatter + Err Bar

In [27]:
fig, axs = plt.subplots(
    figsize=(7.4, 2.0), nrows=1, ncols=4, sharey=True, sharex=True,
    gridspec_kw={"hspace": 0.0, "wspace": 0.0})

for ax in axs.flat:
    ax.grid(True, ls='-', lw=0.25, c='silver')
    ax.tick_params(which='both', direction="in")
    ax.set_xlim(-1.0, 0.6)
    ax.set_ylim(0.2, 0.3)
    ax.set_xticks([-0.8, -0.4, 0, 0.4])
    ax.set_yticks([0.22, 0.24, 0.26, 0.28])
    if ax == axs.flat[0]:
        ax.set_ylabel('median([O/Fe])')
    ax.set_xlabel('median([Fe/H])')
    ax.set_axisbelow(True)

markers = ["o", "v", "^", "d"]
colors = list(settings.component_colors.values())
labels = list(settings.component_labels.values())
for i, ax in enumerate(axs.flat):
    for j in range(4):
        label = labels[j] if j == i else None
        color = colors[j] if j == i else "silver"
        zorder = 10 if j == i else 5
        plot_xerr = xerr[df["Component"] == j].T if j == i else None
        plot_yerr = yerr[df["Component"] == j].T if j == i else None
        ax.errorbar(
            x=df["Fe/H_Median"][df["Component"] == j],
            y=df["O/Fe_Median"][df["Component"] == j],
            xerr=plot_xerr, yerr=plot_yerr,
            c=color, zorder=zorder, linestyle="none",
            ms=3, elinewidth=0.25, marker=None, capsize=0)
        ax.scatter(
            df["Fe/H_Median"][df["Component"] == j],
            df["O/Fe_Median"][df["Component"] == j],
            c=color, label=label, zorder=zorder,
            s=20, linewidths=0.4, edgecolors="white", marker=markers[j])
    ax.legend(loc="lower left", framealpha=0, fontsize=5.0)

fig.savefig("../images/metallicity/OFe_vs_FeH_originals_s127_err.pdf")
plt.close(fig)